<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/power-analysis-via-simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [20]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, 
                          beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      age = runif(sample_size, min = age_lo, max = age_hi)
      x = beta_xa * age + beta_x0 + rnorm(sample_size, mean = 0, sd = sd_x)
      c = beta_ca * age + beta_cx * x + beta_c0 + rnorm(sample_size, mean = 0, sd = sd_c)
      return(data.frame(age, x, c))
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,91.95295,92.902600,347.07355
2,139.03726,3.529762,213.31316
3,110.30536,42.001388,265.25049
4,92.77213,51.318573,261.99855
5,125.11745,58.361804,347.18059
6,80.32904,-34.994286,67.40564


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [48]:
library(mediation)
run_analysis <- function(data) {
    fitM <- lm(x ~ age, data = data)
    fitY <- lm(c ~ x + age, data = data)
    fitMed <- mediate(fitM, fitY, treat = "age", mediator = "x")
    return(c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p))
}
print(run_analysis(dat))

[1] 1.1025977 0.0020000 0.5077538 0.0300000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [49]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(dat)
    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov =  mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [50]:
res <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
res

$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.6

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [53]:
res_50 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 50, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

res_75 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 75, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

res_100 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 100, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) 

res_125 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 125, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)      

res_150 <- repeat_analysis(num_simulations = 10, alpha = .01, sample_size = 150, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) 


Print your results. 

In [54]:
res_df = rbind(res_50,res_75,res_100,res_125,res_150)
print(res_df)

        ACME_cov ADE_cov
res_50  0.8      0.5    
res_75  0.7      0.6    
res_100 1        0.6    
res_125 1        0.9    
res_150 1        0.8    


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> As sample size increases, the coverages also generally increase, with more than .9 coverage reached for both ACME and ADE when the sample size was 125.

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> x is mediator, age is predictor, c is outcome.
When simulating x (word reading experience), x was linked to age first and then comprehension was predicted from both age and x. This made the mediated effect link stronger as the generation of x followed the pattern of mediation. The direct effect of age on comprehension was harder to detect because comprehension was not explicitly generated from only age.

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*